In [413]:
import pandas as pd
import numpy as np

train = pd.read_csv("./dreaddit-train.csv",index_col="id")
test = pd.read_csv("./dreaddit-test.csv",index_col="id")

train.head()

subreddit post_id sentence_range  \
id                                               
33181              ptsd  8601tu       (15, 20)   
2606         assistance  8lbrx9         (0, 5)   
38816              ptsd  9ch1zh       (15, 20)   
239       relationships  7rorpp        [5, 10]   
1421   survivorsofabuse  9p2gbc         [0, 5]   

                                                    text  label  confidence  \
id                                                                            
33181  He said he had not felt that way before, sugge...      1         0.8   
2606   Hey there r/assistance, Not sure if this is th...      0         1.0   
38816  My mom then hit me with the newspaper and it s...      1         0.8   
239    until i met my new boyfriend, he is amazing, h...      1         0.6   
1421   October is Domestic Violence Awareness Month a...      1         0.8   

       social_timestamp  social_karma  syntax_ari  lex_liwc_WC  ...  \
id                                                              ...   
33181        1521614353             5    1.806818          116  ...   
2606         1527009817             4    9.429737          109  ...   
38816        1535935605             2    7.769821          167  ...   
239          1516429555             0    2.667798          273  ...   
1421         1539809005            24    7.554238           89  ...   

       lex_dal_min_pleasantness  lex_dal_min_activation  lex_dal_min_imagery  \
id                                                                             
33181                     1.000                  1.1250                  1.0   
2606                      1.125                  1.0000                  1.0   
38816                     1.000                  1.1429                  1.0   
239                       1.000                  1.1250                  1.0   
1421                      1.000                  1.1250                  1.0   

       lex_dal_avg_activation  lex_dal_avg_imagery  lex_dal_avg_pleasantness  \
id                                                                             
33181                 1.77000              1.52211                   1.89556   
2606                  1.69586              1.62045                   1.88919   
38816                 1.83088              1.58108                   1.85828   
239                   1.75356              1.52114                   1.98848   
1421                  1.77644              1.64872                   1.81456   

       social_upvote_ratio  social_num_comments  syntax_fk_grade  sentiment  
id                                                                           
33181                 0.86                    1         3.253573  -0.002742  
2606                  0.65                    2         8.828316   0.292857  
38816                 0.67                    0         7.841667   0.011894  
239                   0.50                    5         4.104027   0.141671  
1421                  1.00                    1         7.910952  -0.204167  

[5 rows x 115 columns]

In [414]:
print(train.shape)

print(test.shape)

(2838, 115)
(715, 115)


In [415]:
#cat and num segregation

categorical_col = [col for col in train.columns if
                 train[col].dtype == 'object']
numerical_col = [col for col in train.columns if
                 train[col].dtype in ['int64', 'float64']]

print(categorical_col)

print(numerical_col)


['subreddit', 'post_id', 'sentence_range', 'text']
['label', 'confidence', 'social_timestamp', 'social_karma', 'syntax_ari', 'lex_liwc_WC', 'lex_liwc_Analytic', 'lex_liwc_Clout', 'lex_liwc_Authentic', 'lex_liwc_Tone', 'lex_liwc_WPS', 'lex_liwc_Sixltr', 'lex_liwc_Dic', 'lex_liwc_function', 'lex_liwc_pronoun', 'lex_liwc_ppron', 'lex_liwc_i', 'lex_liwc_we', 'lex_liwc_you', 'lex_liwc_shehe', 'lex_liwc_they', 'lex_liwc_ipron', 'lex_liwc_article', 'lex_liwc_prep', 'lex_liwc_auxverb', 'lex_liwc_adverb', 'lex_liwc_conj', 'lex_liwc_negate', 'lex_liwc_verb', 'lex_liwc_adj', 'lex_liwc_compare', 'lex_liwc_interrog', 'lex_liwc_number', 'lex_liwc_quant', 'lex_liwc_affect', 'lex_liwc_posemo', 'lex_liwc_negemo', 'lex_liwc_anx', 'lex_liwc_anger', 'lex_liwc_sad', 'lex_liwc_social', 'lex_liwc_family', 'lex_liwc_friend', 'lex_liwc_female', 'lex_liwc_male', 'lex_liwc_cogproc', 'lex_liwc_insight', 'lex_liwc_cause', 'lex_liwc_discrep', 'lex_liwc_tentat', 'lex_liwc_certain', 'lex_liwc_differ', 'lex_liwc_perce

In [416]:
#EDA

# Using Pandas profiling. But, others like Dtale, Sweetviz, etc, can also be used
# import pandas_profiling as pp
# profile = pp.ProfileReport(train, minimal=True)
# profile.to_file(output_file="report.html")

In [417]:
# different subreddit types
train.subreddit.value_counts()

ptsd                584
relationships       552
anxiety             503
domesticviolence    316
assistance          289
survivorsofabuse    245
homeless            168
almosthomeless       80
stress               64
food_pantry          37
Name: subreddit, dtype: int64

In [418]:
# post id, sentence range aren't useful - using common sense

train = train.drop(['post_id', 'sentence_range'], axis = 1)
test = test.drop(['post_id', 'sentence_range'], axis = 1)

In [419]:
X=train.copy()
y=train.label

X_test=test.copy()
y_test=test.label


In [420]:
# encoding subreddit text column
# the cardinality is 10

from sklearn.preprocessing import OneHotEncoder
print('Unique subreddits in training data {}'.format(len(train['subreddit'].unique())))
print('Unique subreddits in test data {}'.format(len(test['subreddit'].unique())))


OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)

OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train['subreddit'].to_frame()))
OH_cols_test = pd.DataFrame(OH_encoder.transform(test['subreddit'].to_frame()))

OH_cols_train.index = train.index
OH_cols_test.index = test.index

num_X_train = train.drop(['subreddit'], axis=1)
num_X_test = test.drop(['subreddit'], axis=1)

X = pd.concat([num_X_train, OH_cols_train], axis=1)
X_test = pd.concat([num_X_test, OH_cols_test], axis=1)

print(X.head())
print(X_test.head())

Unique subreddits in training data 10
Unique subreddits in test data 10
                                                    text  label  confidence  \
id                                                                            
33181  He said he had not felt that way before, sugge...      1         0.8   
2606   Hey there r/assistance, Not sure if this is th...      0         1.0   
38816  My mom then hit me with the newspaper and it s...      1         0.8   
239    until i met my new boyfriend, he is amazing, h...      1         0.6   
1421   October is Domestic Violence Awareness Month a...      1         0.8   

       social_timestamp  social_karma  syntax_ari  lex_liwc_WC  \
id                                                               
33181        1521614353             5    1.806818          116   
2606         1527009817             4    9.429737          109   
38816        1535935605             2    7.769821          167   
239          1516429555             0    2.6

In [421]:
# remaining text columns

cols=[col for col in X.columns if X[col].dtype=='object']
cols_test=[col for col in X_test.columns if X_test[col].dtype=='object']
print(cols)
print(cols_test)

['text']
['text']


In [422]:
# checking the correlation of features with the TARGET='label'
X_corrs=X.corr().abs()['label'].sort_values(ascending = True)



In [423]:
# text preprocessing helper functions
# Make text lowercase,
# removes hyperlinks,
# remove punctuation
# removes numbers
# tokenizes
# removes stopwords


import nltk,re,string
def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text


def text_preprocessing(text):
    """
    Cleaning and parsing the text.

    """
    tokenizer = nltk.tokenize.RegexpTokenizer(r'\w+')
    nopunc = clean_text(text)
    tokenized_text = tokenizer.tokenize(nopunc)
    combined_text = ' '.join(tokenized_text)
    return combined_text

In [424]:
# Applying the cleaning function to both test and training datasets
X['text_clean'] = X['text'].apply(str).apply(lambda x: text_preprocessing(x))
X_test['text_clean'] = X_test['text'].apply(str).apply(lambda x: text_preprocessing(x))

In [425]:
X['text_len'] = X['text_clean'].astype(str).apply(len)
X['text_word_count'] = X['text_clean'].apply(lambda x: len(str(x).split()))

X_test['text_len'] = X_test['text_clean'].astype(str).apply(len)
X_test['text_word_count'] = X_test['text_clean'].apply(lambda x: len(str(x).split()))

In [426]:
print(X['text_len'].max())
a=[1]
#1599

1599


In [427]:
# for tokenizing
import tensorflow as tf
import transformers

#creating a function
def func_tokenizer(tokenizer_name, docs,max_len=1610):
    ids = []
    masks=[]
    segments=[]
    for doc in docs:
        tokens = tokenizer_name.tokenize(doc)
        tokens=tokens[:max_len-2]
        input_sequence=["[CLS]"]+tokens+["[SEP]"]
        pad_len=max_len-len(input_sequence)
        token_ids = tokenizer_name.convert_tokens_to_ids(input_sequence)
        token_ids+=[0]*pad_len
        pad_masks=[1]*len(input_sequence)+[0]*pad_len
        seg_ids=[0]*max_len
        ids.append(token_ids)
        masks.append(pad_masks)
        segments.append(seg_ids)
        
    return np.array(ids),np.array(masks),np.array(segments)

In [428]:
def text_to_tokenize_bert(X):
    # BERT tokenizer from hugging face

    bert_tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased')

    bert_features = func_tokenizer(bert_tokenizer, X['text_clean'])
    
    ids,masks,segments=bert_features[0],bert_features[1],bert_features[2]

    print('Ids: {}\n\n\n'.format(ids))

    print('Masks: {}\n\n\n'.format(masks))


    print('Segments: {}\n\n\n'.format(segments))
    
    columns_len_tokenzed_data=len(ids[0])


    ids_df=pd.DataFrame(masks,index=X.index,columns=["Tokenized_ID_%d" % (i + 1) for i in range(columns_len_tokenzed_data)])

    masks_df=pd.DataFrame(masks,index=X.index,columns=["Tokenized_Mask_%d" % (i + 1) for i in range(columns_len_tokenzed_data)])

    segments_df=pd.DataFrame(masks,index=X.index,columns=["Tokenized_Segment_%d" % (i + 1) for i in range(columns_len_tokenzed_data)])
    bert_dfs=pd.concat([ids_df,masks_df,segments_df],axis=1)
    
    print(bert_dfs)
    X = X.drop(['text', 'text_clean'], axis = 1)
    X=pd.concat([X,bert_dfs],axis=1)
    return X
    

In [429]:
X=text_to_tokenize_bert(X)
X_test=text_to_tokenize_bert(X_test)


Ids: [[ 101 2002 2056 ...    0    0    0]
 [ 101 4931 2045 ...    0    0    0]
 [ 101 2026 3566 ...    0    0    0]
 ...
 [ 101 1999 2553 ...    0    0    0]
 [ 101 2079 2017 ...    0    0    0]
 [ 101 1045 2001 ...    0    0    0]]



Masks: [[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 ...
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]]



Segments: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]



       Tokenized_ID_1  Tokenized_ID_2  Tokenized_ID_3  Tokenized_ID_4  \
id                                                                      
33181               1               1               1               1   
2606                1               1               1               1   
38816               1               1               1               1   
239                 1               1               1               1   
1421                1               1               1          

In [430]:
#Pre-processed train and test data
print(X.head())
print(X_test.head())

       label  confidence  social_timestamp  social_karma  syntax_ari  \
id                                                                     
33181      1         0.8        1521614353             5    1.806818   
2606       0         1.0        1527009817             4    9.429737   
38816      1         0.8        1535935605             2    7.769821   
239        1         0.6        1516429555             0    2.667798   
1421       1         0.8        1539809005            24    7.554238   

       lex_liwc_WC  lex_liwc_Analytic  lex_liwc_Clout  lex_liwc_Authentic  \
id                                                                          
33181          116              72.64           15.04               89.26   
2606           109              79.08           76.85               56.75   
38816          167              33.80           76.38               86.24   
239            273               2.98           15.25               95.42   
1421            89              3

In [437]:
# writing pre-processed data to file

X.to_csv("pre_processed_train.csv")
X_test.to_csv("pre_processed_test.csv")

In [432]:
# X.drop(["label"], axis = 1, inplace = True)


In [433]:
# y_test=X_test.label
# X_test.drop(["label"], axis = 1, inplace = True)


In [434]:
# from sklearn.metrics import mean_absolute_error
# from sklearn.metrics import mean_squared_error
# from xgboost import XGBClassifier

# gpu_param = {
#     'objective': 'reg:squarederror',
#     'tree_method': 'gpu_hist'
# }
# best_params =  {'n_estimators': 4161, 'max_depth': 2, 'learning_rate': 0.09061477336150617, 'gamma': 0.4, 'min_child_weight': 7, 'subsample': 0.7, 'colsample_bytree': 0.5, 'reg_alpha': 0.7000000000000001, 'reg_lambda': 0.7000000000000001}
# model=XGBClassifier(**best_params,**gpu_param)

In [435]:
# from sklearn.model_selection import RepeatedStratifiedKFold
# def evaluate_model(model, X, y):
# 	cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=4, random_state=1)
# 	scores = cross_val_score(model, X, y, scoring='f1', cv=cv, n_jobs=-1, error_score='raise')
# 	return scores

In [436]:
# from numpy import mean
# from numpy import std
# from sklearn.model_selection import cross_val_score
# scores = evaluate_model(model, X, y)

# print('>%s %.2f (%.2f)' % ("XGB", scores.mean(), std(scores)))